In [1]:
from scipy.io import wavfile
import numpy as np
import torch

# Read WAV file
sample_rate, audio_data = wavfile.read("C:\\Users\\khwez\\Desktop\\Drake Best I Ever Had.wav")

# Convert stereo to mono by averaging the two channels
if len(audio_data.shape) == 2:
    audio_data = np.mean(audio_data, axis=1).astype(audio_data.dtype)

# Define segment length
segment_length = sample_rate  # 1 second

# Calculate total segments and the length of the last segment
total_segments = len(audio_data) // segment_length
remainder = len(audio_data) % segment_length

# Initialize a zero-padded array
padded_audio_data = np.zeros((total_segments + 1) * segment_length, dtype=audio_data.dtype)
padded_audio_data[:len(audio_data)] = audio_data

# Create segments
segments = np.array([padded_audio_data[i:i + segment_length] for i in range(0, len(padded_audio_data), segment_length)])

# Convert to PyTorch tensor
tensor_segments = torch.IntTensor(segments)

# Now tensor_segments is ready to be used in your PyTorch model


In [3]:
from flask import Flask, jsonify, render_template
import random

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/get_data')
def get_data():
    data = {
        "sales": [random.randint(50, 100) for _ in range(7)],
        "inventory": [random.randint(40, 90) for _ in range(7)],
        "profit": [random.randint(30, 80) for _ in range(7)]
    }
    return jsonify(data)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

e:\Studio_Companion\.conda\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
pip install Flask

  Obtaining dependency information for Flask from https://files.pythonhosted.org/packages/fd/56/26f0be8adc2b4257df20c1c4260ddd0aa396cf8e75d90ab2f7ff99bc34f9/flask-2.3.3-py3-none-any.whl.metadata
  Obtaining dependency information for Werkzeug>=2.3.7 from https://files.pythonhosted.org/packages/9b/59/a7c32e3d8d0e546a206e0552a2c04444544f15c1da4a01df8938d20c6ffc/werkzeug-2.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for click>=8.1.3 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/96.1 kB ? eta -:--:--
   ---------------------------------- ----- 81.9/96.1 kB ? eta -:--:--
   ---------------------------------- ----- 81.9/96.1 kB ? eta -:--:--
   ---------------------------------- ----- 81.9/96.1 kB ? eta -:--:--
   ---------------------------------- ----- 81.9/9

In [3]:
pip install librosa --user


  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached audioread-3.0.0-py3-none-any.whl
  Obtaining dependency information for scikit-learn>=0.20.0 from https://files.pythonhosted.org/packages/1b/7c/66d34dc231a309cc3a7f3eeb848319be3b616fd2918db302aaef727eff4a/scikit_learn-1.3.1-cp39-cp39-win_amd64.whl.metadata
  Using cached scikit_learn-1.3.1-cp39-cp39-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for joblib>=0.14 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for numba>=0.51.0 from https://files.pythonhosted.org/packages/2c/31/121c11175e49482ef5

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
from scipy.io import wavfile
import numpy as np
import torch
import librosa
from sklearn.preprocessing import StandardScaler

# Read WAV file
sample_rate, audio_data = wavfile.read("C:\\Users\\khwez\\Desktop\\Drake Best I Ever Had.wav")

# Convert stereo to mono
if len(audio_data.shape) == 2:
    audio_data = np.mean(audio_data, axis=1).astype(audio_data.dtype)

# Ensure audio_data is float32 for librosa
audio_data = audio_data.astype(np.float32)

# Define segment length
segment_length = sample_rate  # 1 second

# Calculate total segments and pad if necessary
total_segments = len(audio_data) // segment_length
remainder = len(audio_data) % segment_length

# Pad audio data with zeros to make it perfectly divisible into segments
if remainder != 0:
    audio_data = np.pad(audio_data, (0, segment_length - remainder), 'constant', constant_values=0)

# Segment audio data
segments = np.array([audio_data[i:i + segment_length] for i in range(0, len(audio_data), segment_length)])

# Initialize feature arrays
mfccs = []
chroma = []
contrast = []

# Extract features for each segment
for segment in segments:
    mfcc = librosa.feature.mfcc(y=segment, sr=sample_rate, n_mfcc=13)
    chroma_stft = librosa.feature.chroma_stft(y=segment, sr=sample_rate)
    spectral_contrast = librosa.feature.spectral_contrast(y=segment, sr=sample_rate)

    mfccs.append(mfcc)
    chroma.append(chroma_stft)
    contrast.append(spectral_contrast)

# Convert list of numpy arrays to a single numpy array
mfccs_np = np.array(mfccs)
chroma_np = np.array(chroma)
contrast_np = np.array(contrast)

# Flatten the array for standardization
mfccs_reshaped = mfccs_np.reshape(-1, mfccs_np.shape[-1])

# Standardize
scaler = StandardScaler()
mfccs_scaled = scaler.fit_transform(mfccs_reshaped)

# Reshape back to original shape
mfccs_scaled = mfccs_scaled.reshape(mfccs_np.shape)

# Convert to PyTorch tensors
tensor_mfccs = torch.FloatTensor(mfccs_scaled)
tensor_chroma = torch.FloatTensor(chroma_np)
tensor_contrast = torch.FloatTensor(contrast_np)

# Save tensors
torch.save(tensor_mfccs, 'tensor_mfccs.pt')


C:\Users\khwez\AppData\Roaming\Python\Python39\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.io import wavfile
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler

# Read WAV file
sample_rate, audio_data = wavfile.read("C:\\Users\\khwez\\Desktop\\Drake Best I Ever Had.wav")

# Convert stereo to mono
if len(audio_data.shape) == 2:
    audio_data = np.mean(audio_data, axis=1).astype(audio_data.dtype)

# Ensure audio_data is float32 for librosa
audio_data = audio_data.astype(np.float32)

# Define segment length
segment_length = sample_rate  # 1 second

# Calculate total segments and pad if necessary
total_segments = len(audio_data) // segment_length
remainder = len(audio_data) % segment_length

# Pad audio data with zeros to make it perfectly divisible into segments
if remainder != 0:
    audio_data = np.pad(audio_data, (0, segment_length - remainder), 'constant', constant_values=0)

# Segment audio data
segments = np.array([audio_data[i:i + segment_length] for i in range(0, len(audio_data), segment_length)])

# Initialize feature arrays
mfccs = []
chroma = []
contrast = []

# Extract features for each segment
for segment in segments:
    mfcc = librosa.feature.mfcc(y=segment, sr=sample_rate, n_mfcc=13)
    chroma_stft = librosa.feature.chroma_stft(y=segment, sr=sample_rate)
    spectral_contrast = librosa.feature.spectral_contrast(y=segment, sr=sample_rate)

    mfccs.append(mfcc)
    chroma.append(chroma_stft)
    contrast.append(spectral_contrast)

# Convert list of numpy arrays to a single numpy array
mfccs_np = np.array(mfccs)
chroma_np = np.array(chroma)
contrast_np = np.array(contrast)

# Flatten the array for standardization
mfccs_reshaped = mfccs_np.reshape(-1, mfccs_np.shape[-1])

# Standardize
scaler = StandardScaler()
mfccs_scaled = scaler.fit_transform(mfccs_reshaped)

# Reshape back to original shape
mfccs_scaled = mfccs_scaled.reshape(mfccs_np.shape)

# Convert to PyTorch tensors
tensor_mfccs = torch.FloatTensor(mfccs_scaled)
tensor_chroma = torch.FloatTensor(chroma_np)
tensor_contrast = torch.FloatTensor(contrast_np)

# Save tensors
torch.save(tensor_mfccs, 'tensor_mfccs.pt')

# Model Architecture
class AudioClassifier(nn.Module):
    def __init__(self):
        super(AudioClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 6 * 6, 128)  # Adjust dimensions based on your MFCC shape
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 10 classes, adjust as per your use-case

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Instantiate the model
model = AudioClassifier()


C:\Users\khwez\AppData\Roaming\Python\Python39\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [9]:
from pydub import AudioSegment
import librosa
import numpy as np

# Load an audio file
audio = AudioSegment.from_file("C:\\Users\\khwez\\Desktop\\Drake Best I Ever Had.wav")

# Resample to 16 kHz
audio = audio.set_frame_rate(16000)

# Define a function to extract features from an audio segment
def extract_features(audio_segment, sample_rate):
    # Convert the audio segment to a numpy array of floating-point values
    audio_data = np.array(audio_segment.get_array_of_samples(), dtype=np.float32)

    # Extract MFCCs (Mel-frequency cepstral coefficients)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)

    # Extract pitch
    pitch, _ = librosa.piptrack(y=audio_data, sr=sample_rate)

    return mfccs, pitch

# Sample usage
sample_rate = 16000  # Adjust to match your audio's sample rate
mfccs, pitch = extract_features(audio, sample_rate)


In [7]:
import librosa
import numpy as np

# Define a function to extract features from an audio segment
def extract_features(audio_segment, sample_rate):
    # Convert the audio segment to a numpy array
    audio_data = np.array(audio_segment.get_array_of_samples())

    # Extract MFCCs (Mel-frequency cepstral coefficients)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)

    # Extract pitch
    pitch, _ = librosa.piptrack(y=audio_data, sr=sample_rate)

    return mfccs, pitch

# Sample usage
sample_rate = 16000  # Adjust to match your audio's sample rate

# Replace 'audio_segment' with the actual audio segment you want to process
mfccs, pitch = extract_features(your_audio_segment, sample_rate)


NameError: name 'your_audio_segment' is not defined

In [1]:
git init


SyntaxError: invalid syntax (3277417328.py, line 1)

In [2]:
pip install pygame


  Obtaining dependency information for pygame from https://files.pythonhosted.org/packages/7f/d9/098baed5e89684f95d042d7d22fc8c0d684748601357cfc884df02bbc43a/pygame-2.5.2-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
    --------------------------------------- 0.2/10.8 MB 5.8 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/10.8 MB 15.6 MB/s eta 0:00:01
   -------------- ------------------------- 4.0/10.8 MB 31.9 MB/s eta 0:00:01
   ----------------------- ---------------- 6.4/10.8 MB 37.3 MB/s eta 0:00:01
   -------------------------------- ------- 8.7/10.8 MB 39.8 MB/s eta 0:00:01
   ---------------------------------------  10.6/10.8 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------  10.8/10.8 MB 43.7 MB/s eta 0:00:01
   ---------------------------------------- 10.8/10.8 MB 34.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pygame

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Ping Pong")
FPS = 60
PADDLE_WIDTH, PADDLE_HEIGHT = 20, 100
BALL_RADIUS = 15
WHITE = (255, 255, 255)

# Initialize variables
paddle_a_y = paddle_b_y = HEIGHT // 2 - PADDLE_HEIGHT // 2
ball_x, ball_y = WIDTH // 2, HEIGHT // 2
ball_dx, ball_dy = 3, 3

clock = pygame.time.Clock()

# Main Loop
run = True
while run:
    clock.tick(FPS)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        paddle_a_y -= 5
    if keys[pygame.K_s]:
        paddle_a_y += 5
    if keys[pygame.K_UP]:
        paddle_b_y -= 5
    if keys[pygame.K_DOWN]:
        paddle_b_y += 5

    # Ball movement
    ball_x += ball_dx
    ball_y += ball_dy

    if ball_y <= 0 or ball_y >= HEIGHT:
        ball_dy *= -1

    if ball_x <= PADDLE_WIDTH and paddle_a_y <= ball_y <= paddle_a_y + PADDLE_HEIGHT or \
       ball_x >= WIDTH - PADDLE_WIDTH and paddle_b_y <= ball_y <= paddle_b_y + PADDLE_HEIGHT:
        ball_dx *= -1

    if ball_x < 0 or ball_x > WIDTH:
        ball_x, ball_y = WIDTH // 2, HEIGHT // 2

    # Draw everything
    WIN.fill((0, 0, 0))
    pygame.draw.rect(WIN, WHITE, (0, paddle_a_y, PADDLE_WIDTH, PADDLE_HEIGHT))
    pygame.draw.rect(WIN, WHITE, (WIDTH - PADDLE_WIDTH, paddle_b_y, PADDLE_WIDTH, PADDLE_HEIGHT))
    pygame.draw.circle(WIN, WHITE, (ball_x, ball_y), BALL_RADIUS)
    pygame.display.update()

pygame.quit()


pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple RPG")
FPS = 30
PLAYER_SIZE = 50
NPC_SIZE = 50
FONT = pygame.font.Font(None, 36)

# Colors
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)

# Initialize variables
player_pos = [WIDTH // 2, HEIGHT // 2]
npc_pos = [100, 100]
quest_status = "Not Started"

clock = pygame.time.Clock()

# Main Loop
while True:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    keys = pygame.key.get_pressed()

    if keys[pygame.K_LEFT]:
        player_pos[0] -= 5
    if keys[pygame.K_RIGHT]:
        player_pos[0] += 5
    if keys[pygame.K_UP]:
        player_pos[1] -= 5
    if keys[pygame.K_DOWN]:
        player_pos[1] += 5

    # NPC interaction: Simple quest
    if pygame.Rect(player_pos[0], player_pos[1], PLAYER_SIZE, PLAYER_SIZE).colliderect(npc_pos[0], npc_pos[1], NPC_SIZE, NPC_SIZE):
        quest_status = "Completed"

    # Drawing
    WINDOW.fill(WHITE)
    pygame.draw.rect(WINDOW, BLUE, (player_pos[0], player_pos[1], PLAYER_SIZE, PLAYER_SIZE))
    pygame.draw.rect(WINDOW, GREEN, (npc_pos[0], npc_pos[1], NPC_SIZE, NPC_SIZE))

    quest_text = FONT.render(f'Quest Status: {quest_status}', True, (0, 0, 0))
    WINDOW.blit(quest_text, (10, 10))

    pygame.display.flip()


SystemExit: 

e:\Studio_Companion\.conda\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple RPG")
FPS = 30
PLAYER_SIZE = 50
NPC_SIZE = 50
FONT = pygame.font.Font(None, 36)

# Colors
WHITE = (255, 255, 255)

# Initialize variables
player_pos = [WIDTH // 2, HEIGHT // 2]
npc_pos = [100, 100]
dialog_box = False

player_img = pygame.image.load('player.png')
npc_img = pygame.image.load('npc.png')

clock = pygame.time.Clock()

# Main Loop
while True:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    keys = pygame.key.get_pressed()

    if keys[pygame.K_LEFT]:
        player_pos[0] -= 5
    if keys[pygame.K_RIGHT]:
        player_pos[0] += 5
    if keys[pygame.K_UP]:
        player_pos[1] -= 5
    if keys[pygame.K_DOWN]:
        player_pos[1] += 5

    # NPC interaction: Dialog box
    if pygame.Rect(player_pos[0], player_pos[1], PLAYER_SIZE, PLAYER_SIZE).colliderect(npc_pos[0], npc_pos[1], NPC_SIZE, NPC_SIZE):
        dialog_box = True
    else:
        dialog_box = False

    # Drawing
    WINDOW.fill(WHITE)
    WINDOW.blit(player_img, (player_pos[0], player_pos[1]))
    WINDOW.blit(npc_img, (npc_pos[0], npc_pos[1]))

    if dialog_box:
        pygame.draw.rect(WINDOW, (0, 0, 0), (100, 500, 600, 100))
        dialog_text = FONT.render('Thank You!', True, WHITE)
        WINDOW.blit(dialog_text, (350, 525))

    pygame.display.flip()


FileNotFoundError: No file 'player.png' found in working directory 'e:\Studio_Companion\src\audio_processing'.

: 

In [1]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple RPG")
FPS = 30
PLAYER_SIZE = 50
NPC_SIZE = 50
FONT = pygame.font.Font(None, 36)

# Colors
WHITE = (255, 255, 255)

# Initialize variables
player_pos = [WIDTH // 2, HEIGHT // 2]
npc_pos = [100, 100]
dialog_box = False

player_img = pygame.image.load('player.png')
npc_img = pygame.image.load('npc.png')

clock = pygame.time.Clock()

# Main Loop
while True:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    keys = pygame.key.get_pressed()

    if keys[pygame.K_LEFT]:
        player_pos[0] -= 5
    if keys[pygame.K_RIGHT]:
        player_pos[0] += 5
    if keys[pygame.K_UP]:
        player_pos[1] -= 5
    if keys[pygame.K_DOWN]:
        player_pos[1] += 5

    # NPC interaction: Dialog box
    if pygame.Rect(player_pos[0], player_pos[1], PLAYER_SIZE, PLAYER_SIZE).colliderect(npc_pos[0], npc_pos[1], NPC_SIZE, NPC_SIZE):
        dialog_box = True
    else:
        dialog_box = False

    # Drawing
    WINDOW.fill(WHITE)
    WINDOW.blit(player_img, (player_pos[0], player_pos[1]))
    WINDOW.blit(npc_img, (npc_pos[0], npc_pos[1]))

    if dialog_box:
        pygame.draw.rect(WINDOW, (0, 0, 0), (100, 500, 600, 100))
        dialog_text = FONT.render('Thank You!', True, WHITE)
        WINDOW.blit(dialog_text, (350, 525))

    pygame.display.flip()


pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: No file 'player.png' found in working directory 'e:\Studio_Companion\src\audio_processing'.

: 

In [1]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple RPG")
FPS = 30
PLAYER_SIZE = 50
NPC_SIZE = 50
FONT = pygame.font.Font(None, 36)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)

# Initialize variables
player_pos = [WIDTH // 2, HEIGHT // 2]
npc_pos = [100, 100]
dialog_box = False

clock = pygame.time.Clock()

# Main Loop
while True:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    keys = pygame.key.get_pressed()

    if keys[pygame.K_LEFT]:
        player_pos[0] -= 5
    if keys[pygame.K_RIGHT]:
        player_pos[0] += 5
    if keys[pygame.K_UP]:
        player_pos[1] -= 5
    if keys[pygame.K_DOWN]:
        player_pos[1] += 5

    # NPC interaction: Dialog box
    if pygame.Rect(player_pos[0], player_pos[1], PLAYER_SIZE, PLAYER_SIZE).colliderect(npc_pos[0], npc_pos[1], NPC_SIZE, NPC_SIZE):
        dialog_box = True
    else:
        dialog_box = False

    # Drawing
    WINDOW.fill(WHITE)
    pygame.draw.rect(WINDOW, BLUE, (player_pos[0], player_pos[1], PLAYER_SIZE, PLAYER_SIZE))
    pygame.draw.rect(WINDOW, GREEN, (npc_pos[0], npc_pos[1], NPC_SIZE, NPC_SIZE))

    if dialog_box:
        pygame.draw.rect(WINDOW, BLACK, (100, 500, 600, 100))
        dialog_text = FONT.render('Thank You!', True, WHITE)
        WINDOW.blit(dialog_text, (350, 525))

    pygame.display.flip()


pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

e:\Studio_Companion\.conda\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Simple RPG")
FPS = 30
PLAYER_SIZE = 50
NPC_SIZE = 50
FONT = pygame.font.Font(None, 36)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)

# Initialize variables
player_pos = [WIDTH // 2, HEIGHT // 2]
npc_pos = [100, 100]
dialog_box = False

clock = pygame.time.Clock()

# Main Loop
while True:
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

    keys = pygame.key.get_pressed()

    if keys[pygame.K_LEFT]:
        player_pos[0] -= 5
    if keys[pygame.K_RIGHT]:
        player_pos[0] += 5
    if keys[pygame.K_UP]:
        player_pos[1] -= 5
    if keys[pygame.K_DOWN]:
        player_pos[1] += 5

    # NPC interaction: Dialog box
    if pygame.Rect(player_pos[0], player_pos[1], PLAYER_SIZE, PLAYER_SIZE).colliderect(npc_pos[0], npc_pos[1], NPC_SIZE, NPC_SIZE):
        dialog_box = True
    else:
        dialog_box = False

    # Drawing
    WINDOW.fill(WHITE)
    pygame.draw.rect(WINDOW, BLUE, (player_pos[0], player_pos[1], PLAYER_SIZE, PLAYER_SIZE))
    pygame.draw.rect(WINDOW, GREEN, (npc_pos[0], npc_pos[1], NPC_SIZE, NPC_SIZE))

    if dialog_box:
        pygame.draw.rect(WINDOW, BLACK, (100, 500, 600, 100))
        dialog_text = FONT.render('Thank You!', True, WHITE)
        WINDOW.blit(dialog_text, (350, 525))

    pygame.display.flip()


SystemExit: 

In [4]:
import requests
import json

def fetch_bing_headlines(api_key):
    url = f"https://api.bing.microsoft.com/v7.0/news/search?q=headlines&count=5&offset=0&mkt=en-US&safeSearch=Moderate"
    headers = {"Ocp-Apim-Subscription-Key": api_key}
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    
    if 'value' in data:
        headlines = [article['name'] for article in data['value'][:5]]
        return headlines
    else:
        return "No headlines found."

api_key = "YOUR_API_KEY"
print(fetch_bing_headlines(api_key))


No headlines found.


In [2]:
pip install pydub


Note: you may need to restart the kernel to use updated packages.


In [8]:
from torch.utils.data import random_split

# Determine dataset size
n_samples = tensor_mfccs.shape[0]

# Define ratios
train_size = int(0.7 * n_samples)
val_size = int(0.15 * n_samples)
test_size = n_samples - train_size - val_size

# Split the dataset
train_set, val_set, test_set = random_split(tensor_mfccs, [train_size, val_size, test_size])


In [2]:
pip install porch 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement porch (from versions: none)
ERROR: No matching distribution found for porch


In [3]:
pip install torch 

     ---------------------------------------- 0.0/172.4 MB ? eta -:--:--
     -------------------------------------- 0.0/172.4 MB 330.3 kB/s eta 0:08:42
     -------------------------------------- 0.1/172.4 MB 787.7 kB/s eta 0:03:39
     ---------------------------------------- 0.2/172.4 MB 1.3 MB/s eta 0:02:12
     ---------------------------------------- 0.3/172.4 MB 2.1 MB/s eta 0:01:22
     ---------------------------------------- 0.8/172.4 MB 3.7 MB/s eta 0:00:47
     ---------------------------------------- 1.7/172.4 MB 6.3 MB/s eta 0:00:28
      --------------------------------------- 2.2/172.4 MB 6.9 MB/s eta 0:00:25
      --------------------------------------- 3.1/172.4 MB 8.6 MB/s eta 0:00:20
      -------------------------------------- 4.3/172.4 MB 10.6 MB/s eta 0:00:16
     - ------------------------------------- 6.1/172.4 MB 13.4 MB/s eta 0:00:13
     - ------------------------------------- 7.5/172.4 MB 15.1 MB/s eta 0:00:11
     - ------------------------------------- 8.